<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       GLMPerSegment and GLMPredictPerSegment Functions in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>The GLM function is used to train the whole data set as one model. The GLMPerSegment function is a partition-by-key function to create a single
    model for each partition. The GLMPredictPerSegment function uses the model generated by
    the GLMPerSegment function to predict target values (regression)
    and class labels (classification) on new input data.
         <br> In this notebook we will see how we can use the GLMPerSegment and GLMPredictPerSegment functions available in Vantage to impute the missing values in dataset.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_GLMPerSegmentandGLMPredictPerSegment_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("decisionforestpredict", ["housing_train", "housing_test"])

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
# Create teradataml DataFrame objects.
housing_train = DataFrame.from_table("housing_train")
housing_test = DataFrame.from_table("housing_test")

In [ ]:
housing_train

In [ ]:
housing_train.shape

In [ ]:
housing_test.shape

<p style = 'font-size:16px;font-family:Arial'>Create GLM model for each stories.<br>
Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(GLMPerSegment)

In [ ]:
# Train the model using the 'Gaussian' family.
GLMPerSegment_out = GLMPerSegment(data=housing_train,
                                    data_partition_column="stories",
                                    input_columns=['garagepl', 'lotsize', 'bedrooms', 'bathrms'],
                                    response_column="price",
                                    family="Gaussian",
                                    iter_max=1000,
                                    batch_size=9)
GLMPerSegment_out.result

<p style = 'font-size:16px;font-family:Arial'>Now lets predict the values using GLMPredictPerSegment function <br>Detailed help can be found by passing function name to built-in help function.</p>

In [ ]:
help(GLMPredictPerSegment)

In [ ]:
# Predict the price using GLMPredictPerSegment().
GLMPredictPerSegment_out = GLMPredictPerSegment(newdata=housing_test,
                                                  newdata_partition_column="stories",
                                                  object=GLMPerSegment_out,
                                                  object_partition_column="stories",
                                                  id_column="sn")

In [ ]:
GLMPredictPerSegment_out.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
db_drop_table("housing_train")

In [ ]:
db_drop_table("housing_test")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>GLMPerSegment function reference: <a href = 'https://docs.teradata.com/search/all?query=GLMPerSegment&content-lang=en-US'>here</a></li>
    <li>GLMPredictPerSegment function reference: <a href = 'https://docs.teradata.com/search/all?query=GLMPredictPerSegment&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>